# ImcPQ

### @Version3 Nov 2022 by Somi Afiuni

### This module provides IMC segmentaion and expressions.

In [69]:
import os
from os import listdir
from glob import glob
import pathlib
import re
import pandas as pd
import numpy as np
from numpy import array
from imctools.converters import ome2analysis
from imctools.converters import ome2histocat
from imctools.converters import mcdfolder2imcfolder
from imctools.converters import exportacquisitioncsv
import skimage
from skimage.transform import rescale, resize
from skimage import exposure
from shutil import copyfile
from skimage.io import imread, imsave
from matplotlib import pyplot as plt
from deepcell.utils.plot_utils import create_rgb_image
from deepcell.applications import Mesmer
from imcpq_exp import get_panel, df_measured

In [82]:
folder_path_base = './analysis'
folder_data_base = './data'
folder_config_base = './config'
folders_path_inputs = ['./data']
input_file_regexp = '.*.zip'
file_path_csv_panel = './config/Panel.csv'
csv_panel_metal = 'MetalTag'
csv_panel_full = 'full'
pathlib.Path(folder_path_base).mkdir(parents=True, exist_ok=True)
pathlib.Path(folder_data_base).mkdir(parents=True, exist_ok=True)
pathlib.Path(folder_config_base).mkdir(parents=True, exist_ok=True)
folder_path_base = pathlib.Path(folder_path_base)
folders_path_inputs = [pathlib.Path(f) for f in folders_path_inputs]
folder_path_analysis = folder_path_base / 'tiffs'
folder_path_ome= folder_path_base / 'ometiff'
folder_path_cp = folder_path_base / 'output'
folder_path_cp_input = folder_path_base / 'output'
suffix_full = '_full'
suffix_mask = '_mask.tiff'
failed_images = list()
for fol in [folder_path_analysis,
            folder_path_ome, folder_path_cp]:
    if not fol.exists():
        fol.mkdir(parents=True)

### Save zipped data (including an mcd file) in ImcPQ > data folder
### Save panel as "Panel" in the ImcPQ> config folder. Panel should include MetalTag, Target, full, and IA columns.

In [72]:
failed_images = list()
re_fn = re.compile(input_file_regexp)

for fol in folders_path_inputs:
    for fn in fol.glob('*'):
        if re_fn.match(fn.name):
            mcdfolder2imcfolder.mcdfolder_to_imcfolder(fn, output_folder=folder_path_ome)
exportacquisitioncsv.export_acquisition_csv(folder_path_ome, output_folder=folder_path_cp_input)
list_analysis_stacks =[
    (csv_panel_full, suffix_full, 0)] #acquisitions[0] index out of range error is because of
            

In [73]:
ome2analysis.omefolder_to_analysisfolder(folder_path_ome, folder_path_analysis, panel_csv_file=file_path_csv_panel,
                                         analysis_stacks=(list_analysis_stacks), metalcolumn=csv_panel_metal)
print(ome2analysis.omefolder_to_analysisfolder)

<function omefolder_to_analysisfolder at 0x7f85806c4670>


## Sample Map

In [74]:
csv_ac_md = os.path.join(folder_path_cp_input, 'acquisition_metadata.csv')
ac_md = pd.read_csv(csv_ac_md, sep=',')
ac_md['TiffName'] = ac_md.AcSession.map(str)+"_s0"+"_a"+ ac_md.id.map(str)+"_ac_full.tiff"
ac_md['ROI_id'] = ac_md.AcSession.map(str) +'_'+ ac_md.id.map(str)
tiffToSample = ac_md[['TiffName', 'description', 'ROI_id']]
tiffToSample = tiffToSample.rename(columns = {'description' : 'Sample', 'ROI_id' : 'SampleName'})
tiffToSample.to_csv('./analysis/output/Sample.csv', index=False)

## Image Normalization 

In [84]:
#image-level arcsinh normalization with cofactor = 5, and then scaling to 0-1 range for each channel
folder_path_analysis = folder_path_base / 'tiffs'
start_dir = folder_path_analysis
for file in os.listdir(start_dir):
    if '_full.tiff' in file:
        full_tiff_file = os.path.join(start_dir, file)
        im = imread(full_tiff_file)
        im = im.astype(np.float64) / 2
        im = np.arcsinh(im)
        img_min = np.min(im, axis=(0, 1), keepdims=True)
        img_max = np.max(im, axis=(0, 1), keepdims=True)
        im = (im - img_min) / (img_max - img_min)
        #convert to uint16 for saving
        # Scale the float64 values to the range [0, 65535]
        scaled_array = np.round(im * 65535).astype(np.uint16)
        # Verify that the scaled values match the original values
       # assert np.allclose(im, scaled_array / 65535) # NEED TO CHECK IF THERE IS STILL SOME LOSS- this checks to see if there is loss... and there is some loss. How can I prevent
        im = scaled_array
        scaled_file_name = file.replace("_full.tiff", "_asin.tiff") #"_asin.tiff"
        fn = os.path.join(start_dir, scaled_file_name)  
        #imsave(fn, im, check_contrast=False)

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [58]:
im.dtype

dtype('float64')

## IA Generating

In [76]:
folder_path_base = pathlib.Path(folder_path_base)
folder_path_analysis = folder_path_base / 'tiffs'
folder_path_base = './analysis'
def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]
csv_tiff_files = find_csv_filenames(folder_path_analysis)
csv_channel_seq = os.path.join(folder_path_analysis, csv_tiff_files[0])
df_ch = pd.read_csv(csv_channel_seq, sep=',', header=None)
df_ch.columns = ['MetalTag']
df = pd.read_csv(file_path_csv_panel, sep=',')
df = df[df['full'] == 1]
join_df = df_ch.set_index('MetalTag').join(df.set_index('MetalTag'))
join_df['metal_ch_index']= range(1, len(join_df) + 1)
IA = list(set(join_df[join_df.IA>0].IA)) 
start_dir = folder_path_analysis

In [77]:
for file in os.listdir(start_dir):
    if '_asin.tiff' in file: #CHANGED
        full_tiff_file = os.path.join(start_dir, file)
        IA_file_name = file.replace("_asin.tiff", "_IA.tiff")
        im = imread(full_tiff_file) 
        print(im.shape)
        c, h, w = im.shape
        if h < c: #the index error happens when number of channels in included in full.tiff is less than number of pixels. 50 is good rule of thumb to avoid this error
            print(h, "height less than number of channels- removed", IA_file_name)
            os.remove(full_tiff_file) #remove file that created error (it is almost always a laser test)
            os.remove(file.replace("_asin.tiff", "_full.csv")) #remove file that created error (it is almost always a laser test)
           # full_tiff_file
            continue
        if w < c:  #the index error happens when number of channels in included in full.tiff is less than number of pixels. 50 is good rule of thumb to avoid this error
            print(w, "width less than number of channels- removed", IA_file_name)
            os.remove(full_tiff_file) #remove file that created error (it is almost always a laser test)
            os.remove(file.replace("_asin.tiff", "_full.csv")) #remove file that created error (it is almost always a laser test)
            continue
        if im.shape[0]<im.shape[1] and im.shape[0]<im.shape[2] : 
            im = np.rollaxis(im, 0, 3) 
            IA_stack = np.zeros((im.shape[0],im.shape[1],3))
            for cats in IA: 
                j = int(cats) 
                pnl = join_df.loc[(join_df['full'] == 1 ) & (join_df['IA'] == j)]
                ims = im[:, :, pnl['metal_ch_index']-1]
                IA_stack[:,:,j-1] = ims.sum(axis=2)
                IA_stack2 = rescale(IA_stack,1, multichannel=True, anti_aliasing=False, mode='reflect')
                IA_stack2 = np.array(IA_stack2,dtype='uint16')
            fn = os.path.join(os.path.abspath(start_dir), IA_file_name)
            imsave(fn, IA_stack2, check_contrast=False)
        

(49, 1000, 1000)
(49, 1000, 1000)
(49, 381, 381)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 944, 1000)
(49, 935, 701)
(49, 373, 373)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 230, 381)
(49, 1000, 1000)
(49, 811, 1000)
(49, 821, 1000)
(49, 1000, 1000)
(49, 797, 1000)
(49, 906, 1000)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 386, 386)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 898, 1000)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 219, 1000)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 435, 1000)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 781, 1000)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 362, 362)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 1000, 1000)
(49, 137, 137)
(49, 731, 1000)
(49, 1000, 1000)
(49,

In [39]:
#pnl['metal_ch_index']-1

## Segmentation

### Add Padding to Smallest Images

In [78]:
#add padding to the smaller images (the ones less than 256), to prevent them from failing segmentation
#all the ones that fail are less than 256 x 256, although not all images less than 256x256 fail
base_path = './analysis/tiffs'
for file in os.listdir(base_path):
    if 'IA.tiff' in file:
        iao = os.path.join(base_path, file)
        im = imread(iao)
        im = im.transpose(2,0,1)  #transpose to work with padding
        c, h, w,  = im.shape
        #print(len(im))
        if h < 256:
            print("Oh No - height too low", file)
            im = np.asarray([np.pad(x, pad_width=((256 - h, 0), (0,0) ), mode='constant', constant_values= 0,) for x in im])
            #print(im.shape)
        #print(w)
        if w < 256:
            print("Oh No - width too low", file) 
            im = np.asarray([np.pad(x, pad_width=((0, 0), (256-w,0) ), mode='constant', constant_values= 0,) for x in im])
            #print(im.shape)
        im = im.transpose(1,2,0)   #transpose back after padding
        #print(im.shape)
        for_tiff = np.array(im, dtype='uint16') 
        fn = os.path.join(base_path, file)  
        imsave(fn, for_tiff, check_contrast=False)

Oh No - width too low TMA_2022_PDAC1_rev_s0_a109_ac_IA.tiff
Oh No - height too low TMA_2022_PDAC1_rev_s0_a112_ac_IA.tiff
Oh No - height too low TMA_2022_PDAC1_rev_s0_a120_ac_IA.tiff
Oh No - width too low TMA_2022_PDAC1_rev_s0_a120_ac_IA.tiff
Oh No - height too low TMA_2022_PDAC1_rev_s0_a57_ac_IA.tiff
Oh No - height too low TMA_2022_PDAC1_rev_s0_a58_ac_IA.tiff
Oh No - width too low TMA_2022_PDAC1_rev_s0_a58_ac_IA.tiff
Oh No - height too low TMA_2022_PDAC1_rev_s0_a2_ac_IA.tiff
Oh No - width too low TMA_2022_PDAC1_rev_s0_a2_ac_IA.tiff


### Deepcell Segmentation

In [79]:
base_path = './analysis/tiffs'
for file in os.listdir(base_path):
    if 'IA.tiff' in file:
        ex_mask_fn = file.replace('IA.tiff', 'IA_mask.tiff')
        if os.path.isfile(os.path.join(base_path, ex_mask_fn)):
            continue
        iao = os.path.join(base_path, file)
        im = imread(iao)
        image = np.expand_dims(im, axis=0)
        rgb_im = create_rgb_image(image[:, :, :, 0:2], channel_colors=['green', 'blue'])
        app = Mesmer()
        seg_pred = app.predict(image[:, :, :, 0:2], image_mpp=0.5)
        for_tiff = np.array(seg_pred[0],dtype='uint16')
        fn = os.path.join(base_path, file.replace('IA.tiff', 'IA_mask.tiff'))
        imsave(fn, for_tiff, check_contrast=False)

/Users/ferris/opt/anaconda3/envs/imcpq_env3/lib/python3.8/site-packages/skimage/morphology/_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


/Users/ferris/opt/anaconda3/envs/imcpq_env3/lib/python3.8/site-packages/skimage/morphology/_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


/Users/ferris/opt/anaconda3/envs/imcpq_env3/lib/python3.8/site-packages/skimage/morphology/_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


/Users/ferris/opt/anaconda3/envs/imcpq_env3/lib/python3.8/site-packages/skimage/morphology/_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


/Users/ferris/opt/anaconda3/envs/imcpq_env3/lib/python3.8/site-packages/skimage/morphology/_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


...

/Users/ferris/opt/anaconda3/envs/imcpq_env3/lib/python3.8/site-packages/skimage/morphology/_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


/Users/ferris/opt/anaconda3/envs/imcpq_env3/lib/python3.8/site-packages/skimage/morphology/_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


/Users/ferris/opt/anaconda3/envs/imcpq_env3/lib/python3.8/site-packages/skimage/morphology/_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


/Users/ferris/opt/anaconda3/envs/imcpq_env3/lib/python3.8/site-packages/skimage/morphology/_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


/Users/ferris/opt/anaconda3/envs/imcpq_env3/lib/python3.8/site-packages/skimage/morphology/_deprecated.py:5: skimage_deprecation: Function ``watershed`` is deprecated and will be removed in version 0.19. Use ``skimage.segmentation.watershed`` instead.
  def watershed(image, markers=None, connectivity=1, offset=None, mask=None,


### Remove Padding

In [85]:
#remove border from masks
#aka, remove the difference between the full tiff file and the IA file
base_path = './analysis/tiffs'
for file in os.listdir(base_path):
    if 'IA_mask.tiff' in file:
        mask_tiff_name = os.path.join(base_path, file)
        full_tiff_name = file.replace('_IA_mask', '_full')
        IA_tiff_name = file.replace('_IA_mask', '_IA')
        
        mask_im = imread(mask_tiff_name)
        full_im = imread(os.path.join(base_path, full_tiff_name))
        IA_im = imread(os.path.join(base_path, IA_tiff_name))
        
        #h_padded, w_padded, c_padded = IA_im.shape
        #h, w, c = full_im.shape
        w_padded, h_padded, c_padded = IA_im.shape
        c, w, h = full_im.shape
       #'need to add a check that channel is in the right place'
        print(mask_tiff_name)
        print(IA_im.shape)
        print(full_im.shape)
        height_to_remove = h_padded - h 
        width_to_remove = w_padded - w
        
        print(height_to_remove)
        print(width_to_remove)

        if height_to_remove != 0:
            mask_im = mask_im[ 0 : w_padded, height_to_remove : h_padded] 
        if width_to_remove != 0:
            mask_im = mask_im[width_to_remove : w_padded, 0: h] 
         
        for_tiff = np.array(mask_im, dtype='uint16') 
        
        print(mask_im.shape)
        imsave(mask_tiff_name, for_tiff, check_contrast=False)

./analysis/tiffs/TMA_2022_PDAC1_rev_s0_a10_ac_IA_mask.tiff
(1000, 1000, 3)
(49, 1000, 1000)
0
0
(1000, 1000, 1)
./analysis/tiffs/TMA_2022_PDAC1_rev_s0_a108_ac_IA_mask.tiff
(1000, 1000, 3)
(49, 1000, 1000)
0
0
(1000, 1000, 1)
./analysis/tiffs/TMA_2022_PDAC1_rev_s0_a57_ac_IA_mask.tiff
(256, 381, 3)
(49, 230, 381)
0
26
(230, 381, 1)
./analysis/tiffs/TMA_2022_PDAC1_rev_s0_a73_ac_IA_mask.tiff
(1000, 1000, 3)
(49, 1000, 1000)
0
0
(1000, 1000, 1)
./analysis/tiffs/TMA_2022_PDAC1_rev_s0_a111_ac_IA_mask.tiff
(1000, 1000, 3)
(49, 1000, 1000)
0
0
(1000, 1000, 1)
./analysis/tiffs/TMA_2022_PDAC1_rev_s0_a34_ac_IA_mask.tiff
(1000, 1000, 3)
(49, 1000, 1000)
0
0
(1000, 1000, 1)
./analysis/tiffs/TMA_2022_PDAC1_rev_s0_a92_ac_IA_mask.tiff
(1000, 1000, 3)
(49, 1000, 1000)
0
0
(1000, 1000, 1)
./analysis/tiffs/TMA_2022_PDAC1_rev_s0_a1_ac_IA_mask.tiff
(381, 381, 3)
(49, 381, 381)
0
0
(381, 381, 1)
./analysis/tiffs/TMA_2022_PDAC1_rev_s0_a123_ac_IA_mask.tiff
(781, 1000, 3)
(49, 781, 1000)
0
0
(781, 1000, 1)
./an

./analysis/tiffs/TMA_2022_PDAC1_rev_s0_a2_ac_IA_mask.tiff
(256, 256, 3)
(49, 123, 123)
133
133
(123, 123, 1)
./analysis/tiffs/TMA_2022_PDAC1_rev_s0_a70_ac_IA_mask.tiff
(1000, 1000, 3)
(49, 1000, 1000)
0
0
(1000, 1000, 1)
./analysis/tiffs/TMA_2022_PDAC1_rev_s0_a112_ac_IA_mask.tiff
(256, 1000, 3)
(49, 219, 1000)
0
37
(219, 1000, 1)
./analysis/tiffs/TMA_2022_PDAC1_rev_s0_a54_ac_IA_mask.tiff
(1000, 1000, 3)
(49, 1000, 1000)
0
0
(1000, 1000, 1)
./analysis/tiffs/TMA_2022_PDAC1_rev_s0_a69_ac_IA_mask.tiff
(1000, 1000, 3)
(49, 1000, 1000)
0
0
(1000, 1000, 1)
./analysis/tiffs/TMA_2022_PDAC1_rev_s0_a88_ac_IA_mask.tiff
(1000, 1000, 3)
(49, 1000, 1000)
0
0
(1000, 1000, 1)
./analysis/tiffs/TMA_2022_PDAC1_rev_s0_a13_ac_IA_mask.tiff
(1000, 1000, 3)
(49, 1000, 1000)
0
0
(1000, 1000, 1)
./analysis/tiffs/TMA_2022_PDAC1_rev_s0_a66_ac_IA_mask.tiff
(1000, 1000, 3)
(49, 1000, 1000)
0
0
(1000, 1000, 1)
./analysis/tiffs/TMA_2022_PDAC1_rev_s0_a104_ac_IA_mask.tiff
(1000, 1000, 3)
(49, 1000, 1000)
0
0
(1000, 1000

## Expressions

In [86]:
#Reading files
current_path = os.path.abspath(os.getcwd())
folder_path_base = os.path.join(current_path, 'analysis', 'tiffs')
folder_data_base = os.path.join(current_path, 'data')
file_path_csv_panel = os.path.join(current_path, 'config', 'Panel.csv') 
file_path_sample = os.path.join(current_path, 'analysis', 'output', 'Sample.csv')
output_measurments_file = os.path.join(current_path, 'analysis', 'output', 'measurements.csv')
sample_df = pd.read_csv(file_path_sample)
panel_df = pd.read_csv(os.path.join(file_path_csv_panel))
panel = get_panel(folder_path_base, panel_df)

In [87]:
expressions = df_measured(folder_path_base, sample_df, panel) #assertion error occurs when I have overwritten the correct matching mask, or when there is no corresponding mask file
expressions.to_csv('./analysis/output/measurements.csv', index=False)